In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="6,7"

In [2]:
%cd ../

/home/qwj/code/HippoRAG


PROMPTS

In [3]:
openie_post_ner_instruction = """Given a question and its entities, extract and list the triplets found in the question. Use placeholders (#1, #2, etc.) for any void items where the specific entity or detail is not provided."""

openie_post_ner_frame = """
Query:
```
When was the institute that owned The Collegian founded?
```
{"named_entities":["The Collegian"]}
"""
ai_output = """
{"triplets": [["The Collegian", "owned by", "#1"], ["#1", "founded", "#2"]]}
"""

real_input_template = """
Query:
```
{passage}
```
{named_entity_json}
"""

def llama3_template(passage, named_entity_json):
    real_input = real_input_template.format(passage = passage, named_entity_json=named_entity_json)
    return "<|begin_of_text|><|start_header_id|>system<|end_header_id|>" + openie_post_ner_instruction + "<|eot_id|>" + \
    "<|start_header_id|>user<|end_header_id|>" + openie_post_ner_frame + "<|eot_id|>" + \
    "<|start_header_id|>assistant<|end_header_id|>" + ai_output + "<|eot_id|>" + \
    "<|start_header_id|>user<|end_header_id|>" + real_input + "<|eot_id|>" + \
     "<|start_header_id|>assistant<|end_header_id|>"

In [4]:
from vllm import LLM, SamplingParams
import pandas as pd

In [5]:
dataset = 'musique'
queries = pd.read_csv(f"output/{dataset}_queries.named_entity_output.tsv", sep='\t')
queries.head()

,Unnamed: 0,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,triples
0,0,2hop__13548_13529,"[{'idx': 0, 'title': 'Lionel Messi', 'paragrap...",When was the person who Messi's goals in Copa ...,"[{'id': 13548, 'question': ""To whom was Messi'...",June 1982,[],True,"{""named_entities"": [""Messi"", ""Copa del Rey"", ""..."
1,1,3hop1__9285_5188_23307,"[{'idx': 0, 'title': 'Member states of NATO', ...",What month did the Tripartite discussions begi...,"[{'id': 9285, 'question': 'What was the nobili...",mid-June,[],True,"{""named_entities"": [""Britain"", ""France"", ""Wars..."
2,2,2hop__766973_770570,"[{'idx': 0, 'title': 'Minsk Region', 'paragrap...",What county is Erik Hort's birthplace a part of?,"[{'id': 766973, 'question': 'Erik Hort >> plac...",Rockland County,"['Rockland County, New York']",True,"{""named_entities"": [""Erik Hort""]}"
3,3,2hop__170823_120171,"[{'idx': 0, 'title': 'Blast Corps', 'paragraph...",What year did the publisher of Labyrinth end?,"[{'id': 170823, 'question': 'Labyrinth >> publ...",1986,[],True,"{""named_entities"": [""Labyrinth""]}"
4,4,2hop__511454_120259,"[{'idx': 0, 'title': 'Kavangoland', 'paragraph...",When was Lady Godiva's birthplace abolished?,"[{'id': 511454, 'question': 'Lady Godiva >> pl...",918,[],True,"{""named_entities"": [""Lady Godiva's birthplace""]}"


In [6]:
prompts = []
for i, row in queries.iterrows():
    passage = row.get('query', row.get('question'))
    named_entity_json = row['triples']
    prompts.append(llama3_template(passage, named_entity_json))
len(prompts)

1000

In [7]:
print(prompts[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>Given a question and its entities, extract and list the triplets found in the question. Use placeholders (#1, #2, etc.) for any void items where the specific entity or detail is not provided.<|eot_id|><|start_header_id|>user<|end_header_id|>
Query:
```
When was the institute that owned The Collegian founded?
```
{"named_entities":["The Collegian"]}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{"triplets": [["The Collegian", "owned by", "#1"], ["#1", "founded", "#2"]]}
<|eot_id|><|start_header_id|>user<|end_header_id|>
Query:
```
When was the person who Messi's goals in Copa del Rey compared to get signed by Barcelona?
```
{"named_entities": ["Messi", "Copa del Rey", "Barcelona"]}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>


## Inference

In [8]:
model = LLM(model="/data/qwj/model/llama3_8b_instruct_hf", tensor_parallel_size=2, enable_prefix_caching=True)
sampling_params = SamplingParams(
    n=1, stop=['<|eot_id|>'], max_tokens=1000
)


2024-07-24 12:31:37,267	INFO worker.py:1749 -- Started a local Ray instance.


INFO 07-24 12:31:38 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/data/qwj/model/llama3_8b_instruct_hf', speculative_config=None, tokenizer='/data/qwj/model/llama3_8b_instruct_hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/data/qwj/model/llama3_8b_instruct_hf)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-24 12:31:41 utils.py:660] Found nccl from library /home/qwj/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=2894003) INFO 07-24 12:31:41 utils.py:660] Found nccl from library /home/qwj/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=2894003) INFO 07-24 12:31:42 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
(RayWorkerWrapper pid=2894003) INFO 07-24 12:31:42 selector.py:32] Using XFormers backend.
INFO 07-24 12:31:42 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 07-24 12:31:42 selector.py:32] Using XFormers backend.
INFO 07-24 12:31:43 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=2894003) INFO 07-24 12:31:43 pynccl_utils.py:43] vLLM is using nccl==2.18.1
INFO 07-24 12:31:44 utils.py:132] reading GPU P2P access cache from /home/qwj/.config/

(RayWorkerWrapper pid=2894003) INFO 07-24 12:31:57 model_runner.py:1017] Graph capturing finished in 6 secs.


In [9]:
outputs = model.generate(prompts[:10], sampling_params)
output_text = [o.outputs[0].text for o in outputs]

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 10/10 [00:01<00:00,  8.83it/s]


In [10]:
output_text

['\n\n{"triplets": [["Messi", "goals in", "Copa del Rey"], ["#1", "compared to", "#2"], ["#2", "get signed by", "Barcelona"]]}',
 '\n\n{"triplets": [["Britain", "participated in", "Tripartite discussions"], ["France", "participated in", "Tripartite discussions"], ["#1", "originated from", "Warsaw Pact"]]}',
 '\n\n{"triplets": [["Erik Hort", "birthplace", "#1"], ["#1", "part of", "#2"]]}',
 '\n\n{"triplets": [["Labyrinth", "published by", "#1"], ["#1", "ended", "#2"]]}',
 '\n\n{"triplets": [["Lady Godiva\'s birthplace", "abolished", "#1"]]}',
 '\n\n{"triplets": [["Israel", "located in", "#1"], ["#1", "north of", "#2"], ["#2", "location of", "#3"]]}',
 '\n\n{"triplets": [["#1", "body of water", "Southeast Library"], ["Southeast Library", "located in", "#2"], ["#2", "empties into", "Gulf of Mexico"]]}',
 '\n\n{"triplets": [["explorer", "reach", "city"], ["city", "headquarters of", "#1"], ["#1", "located", "#2"]]}',
 '\n\n{"triplets": [["performer of III", "birthplace", "#1"], ["#1", "star

## Extract relations from results

In [11]:
import re
import json

def extract_json_dict(text):
    pattern = r'\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\})*)*\})*)*\}'
    match = re.search(pattern, text)

    if match:
        json_string = match.group()
        try:
            json_dict = json.loads(json_string)
            return json_dict
        except json.JSONDecodeError:
            return text
    else:
        return text

In [12]:
for i, t in enumerate(output_text):
    print(i, extract_json_dict(t))

0 {'triplets': [['Messi', 'goals in', 'Copa del Rey'], ['#1', 'compared to', '#2'], ['#2', 'get signed by', 'Barcelona']]}
1 {'triplets': [['Britain', 'participated in', 'Tripartite discussions'], ['France', 'participated in', 'Tripartite discussions'], ['#1', 'originated from', 'Warsaw Pact']]}
2 {'triplets': [['Erik Hort', 'birthplace', '#1'], ['#1', 'part of', '#2']]}
3 {'triplets': [['Labyrinth', 'published by', '#1'], ['#1', 'ended', '#2']]}
4 {'triplets': [["Lady Godiva's birthplace", 'abolished', '#1']]}
5 {'triplets': [['Israel', 'located in', '#1'], ['#1', 'north of', '#2'], ['#2', 'location of', '#3']]}
6 {'triplets': [['#1', 'body of water', 'Southeast Library'], ['Southeast Library', 'located in', '#2'], ['#2', 'empties into', 'Gulf of Mexico']]}
7 {'triplets': [['explorer', 'reach', 'city'], ['city', 'headquarters of', '#1'], ['#1', 'located', '#2']]}
8 {'triplets': [['performer of III', 'birthplace', '#1'], ['#1', 'start of battle', '#2']]}
9 {'triplets': [['Till dom ensa

In [13]:
outputs = model.generate(prompts, sampling_params)
extracted_triples = [extract_json_dict(o.outputs[0].text) for o in outputs]

Processed prompts: 100%|██████████| 1000/1000 [00:36<00:00, 27.63it/s]


In [14]:
queries.columns

Index(['Unnamed: 0', 'id', 'paragraphs', 'question', 'question_decomposition',
       'answer', 'answer_aliases', 'answerable', 'triples'],
      dtype='object')

In [15]:
data_to_save = queries.copy()[['Unnamed: 0', 'id', 'question','triples']]
data_to_save['extracted_triples'] = extracted_triples
data_to_save.to_csv(f"output/{dataset}_queries.triples.tsv", sep='\t')

In [16]:
data_to_save.head(10)

,Unnamed: 0,id,question,triples,extracted_triples
0,0,2hop__13548_13529,When was the person who Messi's goals in Copa ...,"{""named_entities"": [""Messi"", ""Copa del Rey"", ""...","{'triplets': [['#1', 'compared by', 'Messi'], ..."
1,1,3hop1__9285_5188_23307,What month did the Tripartite discussions begi...,"{""named_entities"": [""Britain"", ""France"", ""Wars...","{'triplets': [['the country where', 'the top-r..."
2,2,2hop__766973_770570,What county is Erik Hort's birthplace a part of?,"{""named_entities"": [""Erik Hort""]}","{'triplets': [['Erik Hort', 'birthplace', '#1'..."
3,3,2hop__170823_120171,What year did the publisher of Labyrinth end?,"{""named_entities"": [""Labyrinth""]}","{'triplets': [['Labyrinth', 'published by', '#..."
4,4,2hop__511454_120259,When was Lady Godiva's birthplace abolished?,"{""named_entities"": [""Lady Godiva's birthplace""]}","{'triplets': [['Lady Godiva's birthplace', 'ab..."
5,5,4hop2__71753_648517_70784_79935,When was the region immediately north of the r...,"{""named_entities"": [""Israel"", ""Battle of Qurah...","{'triplets': [['#1', 'is located in', 'region ..."
6,6,4hop1__94201_642284_131926_89261,Where does the body of water by the city where...,"{""named_entities"": [""Southeast Library"", ""Gulf...",{'triplets': [['body of water by the city wher...
7,7,4hop1__152562_5274_458768_33633,When did the explorer reach the city where the...,"{""named_entities"": [""Vilaiyaadu Mankatha""]}","{'triplets': [['#1', 'reached', '#2'], ['#2', ..."
8,8,3hop1__305282_282081_73772,When was the start of the battle of the birthp...,"{""named_entities"": [""battle of the birthplace""...","{'triplets': [['performer of III', 'performer ..."
9,9,2hop__655505_110949,What is the Till dom ensamma performer's birth...,"{""named_entities"": [""Till dom ensamma""]}","{'triplets': [['Till dom ensamma', 'has birth ..."


In [ ]:
def build_tree